In [23]:
# I'm using Google Drive account from the Colab environment by importing the Google Colab library.

from google.colab import drive

from tensorflow.keras import layers, Sequential, Input, Model
#importing the callbacks that will be used to save our model checkpoints and halt training early if necessary.
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing import image_dataset_from_directory

import numpy as np
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import Resize, Normalize, ToTensor, Compose, RandomVerticalFlip, RandomHorizontalFlip, RandomPerspective, RandomInvert, RandomAutocontrast
#Setting the path to the directory where our dataset is stored in our Google Drive account.
drive.mount('/content/drive')
dataset_path = '/content/drive/MyDrive/fake_image_detection'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).



image_size is a tuple specifying the desired size of the images default is (256,256)
batch_size is an integer representing the size of the batches to use during training default is 32.


In [24]:
def get_dataset_split_tensorflow(directory, image_size=(256,256), batch_size=32):
  train_data  = image_dataset_from_directory(directory=f'{dataset_path}/train', label_mode='binary', image_size=image_size, batch_size=batch_size)
  val_data    = image_dataset_from_directory(directory=f'{dataset_path}/valid', label_mode='binary', image_size=image_size, batch_size=batch_size)
  test_data   = image_dataset_from_directory(directory=f'{dataset_path}/test', label_mode='binary', image_size=image_size, batch_size=batch_size)

  return train_data, val_data, test_data

In [25]:
train_data_tensorflow, val_data_tensorflow, test_data_tensorflow = get_dataset_split_tensorflow(dataset_path)

Found 189 files belonging to 2 classes.
Found 21 files belonging to 2 classes.
Found 90 files belonging to 2 classes.


In [26]:
# RandomFlip: flips the image randomly either horizontally or vertically.
# RandomTranslation: Flips the image randomly, either horizontally or vertically.
# RandomRotation: Within a given range, RandomRotation rotates the image at random.
# RandomZoom: randomly increases the image's magnification within a certain range.
# Rescaling: When rescaling, the image's pixel values are adjusted to fall between [0,1].

data_augmentation = Sequential([layers.RandomFlip("horizontal_and_vertical"), layers.RandomTranslation(0.3,0.3), layers.RandomRotation(0.4), layers.RandomZoom(0.4), layers.Rescaling(1./255)])


The following functions define metrics for evaluating the performance of the model. 
get_recall() calculates the recall score, 
get_precision() calculates the precision score, and 
get_f1() calculates the F1 score, which is the harmonic mean of precision and recall. 
These functions use the Keras backend to perform the calculations.


In [27]:
from keras import backend as K

def get_recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def get_precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def get_f1(y_true, y_pred):
    precision = get_precision(y_true, y_pred)
    recall = get_recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


The model consists of 4 convolutional layers with increasing number of filters(16, 32, 64, and 64 respectively),
followed by batch normalization, ReLU activation function and max-pooling layer. Initialization techniques such as he_uniform and zeros biases help initialize the weights and batch normalization helps reduce training time and increase accuracy.


In [28]:
from keras import initializers

def CNNmodel(input_shape):
    inputs = Input(shape=input_shape+(3,))
    x = data_augmentation(inputs)

    # 1st Convolutional Layer
    x = layers.Conv2D(16, 3, strides = 2, kernel_initializer = 'he_uniform',
    bias_initializer = initializers.Zeros(), padding = 'same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D(3, strides = 1, padding = 'same')(x)

    # 2nd Convolutional Layer
    x = layers.Conv2D(32, 3, strides = 2, kernel_initializer = 'he_uniform',
    bias_initializer = initializers.Zeros(), padding = 'same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D(3, strides = 1, padding = 'same')(x)

    # 3rd Convolutional Layer
    x = layers.Conv2D(64, 3, strides = 2, kernel_initializer = 'he_uniform',
    bias_initializer = initializers.Zeros(), padding = 'same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D(3, strides = 1, padding = 'same')(x)

    # 4th Convolutional Layer
    x = layers.Conv2D(64, 3, strides = 2, kernel_initializer = 'he_uniform',
    bias_initializer = initializers.Zeros(), padding = 'same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D(3, strides = 1, padding='same')(x)

    # Passing it to a Fully Connected layer
    x = layers.Flatten()(x)

    # 1st Fully Connected Layer
    x = layers.Dense(256, kernel_initializer = 'he_uniform',
    bias_initializer=initializers.Zeros())(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    # Output Layer
    outputs = layers.Dense(1, activation="sigmoid")(x)
    return Model(inputs, outputs)


model = CNNmodel(input_shape=(256,256))
model.summary()

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 256, 256, 3)]     0         
                                                                 
 sequential_1 (Sequential)   (None, 256, 256, 3)       0         
                                                                 
 conv2d_32 (Conv2D)          (None, 128, 128, 16)      448       
                                                                 
 batch_normalization_40 (Bat  (None, 128, 128, 16)     64        
 chNormalization)                                                
                                                                 
 activation_40 (Activation)  (None, 128, 128, 16)      0         
                                                                 
 max_pooling2d_32 (MaxPoolin  (None, 128, 128, 16)     0         
 g2D)                                                      


Now, i'm going to set the number of epochs to 1, by choosing the Adam optimizer and setting the loss function to binary cross-entropy. Then, the model is compiled with the optimizer, loss function, and metrics to evaluate during training (accuracy, F1 score, precision, and recall). 
Finally, the model is trained on the train_data_tensorflow dataset for the specified number of epochs, and the validation set is used to evaluate the model after each epoch. The training history is stored in the history variable for later analysis.


In [29]:
from tensorflow.keras import optimizers

epochs = 1
optimizer = optimizers.Adam()
loss = "binary_crossentropy"

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy',get_f1, get_precision, get_recall])
history = model.fit(train_data_tensorflow, epochs=epochs, validation_data=val_data_tensorflow)

6/6 [==============================] - 25s 2s/step - loss: 1.5743 - accuracy: 0.4286 - get_f1: 0.3855 - get_precision: 0.4212 - get_recall: 0.3611 - val_loss: 13.1452 - val_accuracy: 0.4762 - val_get_f1: 0.6452 - val_get_precision: 0.4762 - val_get_recall: 1.0000



Evaluating the performance of the trained model on the test dataset. 
It also calculates the loss, accuracy, f1 score, precision, and recall on the test dataset using the evaluate() method of the trained model. 
Atlast it prints the test accuracy, test recall, and test f1_score rounded to 4 decimal places using the print() function.


In [30]:
test_loss, test_acc, test_f1, test_precision, test_recall = model.evaluate(test_data_tensorflow)
print('\nTest accuracy:', round(test_acc*100,4))
print('Test recall:', round(test_recall*100,4))
print('Test f1_score:', round(test_f1*100,4))

3/3 [==============================] - 1s 236ms/step - loss: 11.3481 - accuracy: 0.5000 - get_f1: 0.6618 - get_precision: 0.4976 - get_recall: 1.0000

Test accuracy: 50.0
Test recall: 100.0
Test f1_score: 66.1791


# Tuning the CNN model

ResNet function creates a model based on the ResNet50 architecture with some modifications. 
The ResNet50 model is first loaded with pre-trained ImageNet weights and then made non-trainable. 
It takes this pre-trained ResNet50 model as input and adds some fully connected layers on top of it to create the final model. It prints the summary of tuned model.

In [31]:
from tensorflow.keras import applications, regularizers

def ResNet(pre_trained, input_shape):
  inputs = Input(shape=input_shape+(3,))
  x = data_augmentation(inputs)
  x = pre_trained(x)
  x = layers.Flatten()(x)
  
  x = layers.Dropout(0.5)(x) 
  x = layers.Dense(1024, activation="relu")(x) 
  x = layers.Dense(512, activation="relu")(x)  
  x = layers.Dense(128, activation="relu")(x) 
  output = layers.Dense(1, activation='sigmoid')(x)
  return Model(inputs, output)

pre_trained = applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=(256,256,3),
)
    
pre_trained.trainable = False

tuned_model = ResNet(pre_trained, input_shape=(256,256))
tuned_model.summary()

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 256, 256, 3)]     0         
                                                                 
 sequential_1 (Sequential)   (None, 256, 256, 3)       0         
                                                                 
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 flatten_10 (Flatten)        (None, 131072)            0         
                                                                 
 dropout_1 (Dropout)         (None, 131072)            0         
                                                                 
 dense_22 (Dense)            (None, 1024)              134218752 
                                                                 
 dense_23 (Dense)            (None, 512)               524


defining an optimizer, loss function, and compiles a pre-trained ResNet50 model for binary classification. It then trains the model for 10 epochs using the specified optimizer and loss function on training data, and evaluates its performance on validation data.


In [32]:
from tensorflow.keras import optimizers
from tensorflow.keras import losses

epochs = 10
optimizer = optimizers.Adam(learning_rate=0.01)
loss = losses.BinaryCrossentropy(from_logits=False)

tuned_model.compile(optimizer=optimizer, loss=loss, metrics=['acc', get_f1, get_precision, get_recall])
history = tuned_model.fit(train_data_tensorflow, epochs=epochs, validation_data=val_data_tensorflow)

Epoch 1/10
6/6 [==============================] - 80s 12s/step - loss: 823.6858 - acc: 0.3915 - get_f1: 0.2810 - get_precision: 0.1979 - get_recall: 0.5000 - val_loss: 52.1271 - val_acc: 0.5238 - val_get_f1: 0.0000e+00 - val_get_precision: 0.0000e+00 - val_get_recall: 0.0000e+00
Epoch 2/10
6/6 [==============================] - 71s 12s/step - loss: 27.7219 - acc: 0.4762 - get_f1: 0.2126 - get_precision: 0.1562 - get_recall: 0.3333 - val_loss: 60.0396 - val_acc: 0.4762 - val_get_f1: 0.6452 - val_get_precision: 0.4762 - val_get_recall: 1.0000
Epoch 3/10
6/6 [==============================] - 70s 12s/step - loss: 50.7070 - acc: 0.4868 - get_f1: 0.4349 - get_precision: 0.3281 - get_recall: 0.6667 - val_loss: 28.1774 - val_acc: 0.5238 - val_get_f1: 0.0000e+00 - val_get_precision: 0.0000e+00 - val_get_recall: 0.0000e+00
Epoch 4/10
6/6 [==============================] - 71s 12s/step - loss: 14.7175 - acc: 0.5556 - get_f1: 0.3575 - get_precision: 0.2816 - get_recall: 0.5000 - val_loss: 11.0460


Evaluating the performance of the tuned_model on the test dataset and prints the accuracy, recall, and F1 score.


In [33]:
test_loss_tuned, test_acc_tuned, test_f1_tuned, test_precision_tuned, test_recall_tuned = tuned_model.evaluate(test_data_tensorflow)
print('\nTest accuracy:', round(test_acc_tuned*100,4))
print('Test recall:', round(test_recall_tuned*100,4))
print('Test f1_score:', round(test_f1_tuned*100,4))

3/3 [==============================] - 24s 7s/step - loss: 0.8353 - acc: 0.5000 - get_f1: 0.6688 - get_precision: 0.5072 - get_recall: 1.0000

Test accuracy: 50.0
Test recall: 100.0
Test f1_score: 66.8783



Training a CNN model with different combinations of optimizers and regularizers and evaluates them on a test dataset. The early stopping callback is used to stop training if validation loss does not improve.


In [34]:
from keras import optimizers, regularizers
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    verbose=1,
    mode='min',
    restore_best_weights=True
)

learning_rate = 0.0001
batch_size = 32
num_epoch = 5
optimizers = [optimizers.SGD(), optimizers.Adam()]
regularizers = [regularizers.l1(0.01), regularizers.l2(0.01), regularizers.l1_l2(l1=0.01, l2=0.01)]


for opt in optimizers:
    for reg in regularizers:
        model = CNNmodel(input_shape = (256, 256))
        model.compile(
            optimizer = opt,
            loss = 'binary_crossentropy',
            metrics = ['accuracy']
        )

        history = model.fit(
            train_data_tensorflow,
            epochs = num_epoch,
            batch_size = batch_size,
            validation_data = val_data_tensorflow,
            callbacks = [early_stopping],
            verbose = 1
        )
        # Evaluate the model
        loss, acc = model.evaluate(test_data_tensorflow)
        print(f'\nOptimizer: {opt}, \nRegularizer: {reg}, \n loss: {loss}, \n accuracy: {acc}\n')


Epoch 1/5
6/6 [==============================] - 15s 2s/step - loss: 0.8885 - accuracy: 0.4762 - val_loss: 1.2168 - val_accuracy: 0.4762
Epoch 2/5
6/6 [==============================] - 13s 2s/step - loss: 0.8210 - accuracy: 0.4868 - val_loss: 0.7177 - val_accuracy: 0.4762
Epoch 3/5
6/6 [==============================] - 13s 2s/step - loss: 0.8455 - accuracy: 0.4762 - val_loss: 0.8215 - val_accuracy: 0.4762
Epoch 4/5
6/6 [==============================] - 13s 2s/step - loss: 0.7575 - accuracy: 0.5132 - val_loss: 0.6305 - val_accuracy: 0.6667
Epoch 5/5
3/3 [==============================] - 1s 220ms/step - loss: 0.8452 - accuracy: 0.4889

Optimizer: <keras.optimizers.legacy.gradient_descent.SGD object at 0x7f8c445b9490>, 
Regularizer: <keras.regularizers.L1 object at 0x7f8c445b96d0>, 
 loss: 0.8452138304710388, 
 accuracy: 0.4888888895511627

Epoch 1/5
6/6 [==============================] - 15s 2s/step - loss: 0.8325 - accuracy: 0.4815 - val_loss: 0.6840 - val_accuracy: 0.5238
Epoch 2/5

Optimizer: optimizers.Adam()

Regularizer: regularizers.l1_l2(l1=0.01, l2=0.01)

Loss: 1.2607940435409546 

Accuracy: 0.5444444417953491